# RYR3 Mutation Trans Effect on Transcriptomics

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings("ignore")

import cptac
import cptac.utils as al

brain = cptac.Gbm()

### Select Gene

In [2]:
gene = "RYR3"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
omics = "transcriptomics"
#omics = "phosphoproteomics"
#omics = "acetylproteomics"

### Track all significant comparisons in Dataframe

In [4]:
all_significant_comparisons = pd.DataFrame(columns=['Cancer_Type', 'Gene', 'Comparison','Interacting_Protein','P_Value'])

In [5]:
def add_to_all_significant_comparisons(df, cancer, interacting, all_sig_comp):
    expanded = df
    expanded['Gene'] = gene
    expanded['Cancer_Type'] = cancer
    expanded['Interacting_Protein'] = interacting
    
    updated_all_comparisons = pd.concat([all_sig_comp, expanded], sort=False)
    
    return updated_all_comparisons

# Interacting Proteins: Transcriptomics

### Generate interacting protein list

Make a call to cptac.utils to get the interacting proteins method, which interacts with the uniprot and string databases to generate a list of known interacting partners with the given gene.

In [6]:
# Use get interacting proteins method to generate list of interacting proteins
interacting_proteins = al.get_interacting_proteins(gene)

# Show interacting protein list
print("Interacting Proteins:")
for interacting_protein in interacting_proteins:
    print(interacting_protein)

Interacting Proteins:
CACNA1C
SRI
CALM2
CALM3
CALML6
CALML3
CALM1
ATP2A1
RYR1
CD38
ATP2A3
SEPN1
CACNA1S
RYR2
PRKG1
ASPH
CALML5
FKBP1B
RYR3
WBSCR17
ARHGAP11A
FKBP1A
CALML4
TRDN
ATP2A2
NOS1
CUL1
CFTR
HNF1A
BMI1
VCP


### Test for significant comparisons in any of interacting proteins

In [7]:
# Create dataframe in order to do comparisons with wrap_ttest
protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins)
protdf = brain.reduce_multiindex(protdf, flatten=True)
protdf = protdf[protdf.Sample_Status == 'Tumor'] # drop Normal samples

# Create the binary valued column needed to do the comparison
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

# Format the dataframe correctly
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


# Make list of columns to be compared using t-tests
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

# Call wrap_ttest, pass in formatted dataframe
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

# Print results, if anything significant was found
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", True, all_significant_comparisons)
        

Doing t-test comparisons

No significant comparisons.


# All Proteins: Transcriptomics 

In [8]:
try:
    print("\nGene: ", gene)

    # Use all proteins

    # Create dataframe in order to do comparisons with wrap_ttest
    protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics)
    protdf = brain.reduce_multiindex(protdf, flatten=True) # flatten for specific column names
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    # Create binary column 
    protdf['Label'] = np.where(
                protdf[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

    # Format the dataframe correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    # Make list of columns to be compared using t-tests
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    # Call wrap_ttest, pass in formatted dataframe
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    # Print results, if anything significant was found
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")
            
            all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", False, all_significant_comparisons)


except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  RYR3


Doing t-test comparisons

                                         Comparison       P_Value
0         MIR4309_transcriptomics_ENSG00000266015.1  2.141199e-08
1    ARHGEF38-IT1_transcriptomics_ENSG00000249885.1  1.447441e-07
2       LINC01047_transcriptomics_ENSG00000232225.5  1.966811e-07
3       RNU6-227P_transcriptomics_ENSG00000199735.1  2.400447e-07
4     RP1-59B16.1_transcriptomics_ENSG00000220748.2  3.271365e-07
5        SEPHS1P7_transcriptomics_ENSG00000231926.1  3.271541e-07
6   RP11-551G24.3_transcriptomics_ENSG00000236775.1  3.273002e-07
7       MIR5701-1_transcriptomics_ENSG00000266545.1  3.273574e-07
8        TCEB1P31_transcriptomics_ENSG00000256021.1  3.273574e-07
9       RNU6-210P_transcriptomics_ENSG00000252752.1  3.289980e-07
10      SIGLEC31P_transcriptomics_ENSG00000268847.1  3.289980e-07
11        MIR4288_transcriptomics_ENSG00000265251.1  3.322928e-07
12          Y_RNA_transcriptomics_ENSG00000206847.1  3.330977e-07
13   RP11-316I3.2_transcriptomics_ENSG00000238054.

# Print all significant comparisons

In [9]:
if len(all_significant_comparisons) > 0:
    display(all_significant_comparisons)
    
else:
    print('No Significant Comparisons!')

,Cancer_Type,Gene,Comparison,Interacting_Protein,P_Value
0,Gbm,RYR3,MIR4309_transcriptomics_ENSG00000266015.1,False,2.141199e-08
1,Gbm,RYR3,ARHGEF38-IT1_transcriptomics_ENSG00000249885.1,False,1.447441e-07
2,Gbm,RYR3,LINC01047_transcriptomics_ENSG00000232225.5,False,1.966811e-07
3,Gbm,RYR3,RNU6-227P_transcriptomics_ENSG00000199735.1,False,2.400447e-07
4,Gbm,RYR3,RP1-59B16.1_transcriptomics_ENSG00000220748.2,False,3.271365e-07
5,Gbm,RYR3,SEPHS1P7_transcriptomics_ENSG00000231926.1,False,3.271541e-07
6,Gbm,RYR3,RP11-551G24.3_transcriptomics_ENSG00000236775.1,False,3.273002e-07
7,Gbm,RYR3,MIR5701-1_transcriptomics_ENSG00000266545.1,False,3.273574e-07
8,Gbm,RYR3,TCEB1P31_transcriptomics_ENSG00000256021.1,False,3.273574e-07
9,Gbm,RYR3,RNU6-210P_transcriptomics_ENSG00000252752.1,False,3.289980e-07


### Write Significant Comparisons (if any) to Shared CSV file

In [10]:
'''
existing_results = pd.read_csv(gene+'_Trans_Results.csv')

updated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)

updated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)'''

"\nexisting_results = pd.read_csv(gene+'_Trans_Results.csv')\n\nupdated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)\n\nupdated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)"